# 2.Search Engine

In [57]:
import nltk
from nltk.stem import PorterStemmer

In [58]:
def text_mining(synopsis):
    tokens = nltk.word_tokenize(synopsis)                           #tokenization
    tokens = [word for word in tokens if word.isalpha()]            #remove punctuations and numbers
    stop_words = set(nltk.corpus.stopwords.words('english'))        #remove stopwords
    tokens = [word for word in tokens if not word.lower() in stop_words]       
    tokens = [PorterStemmer().stem(word) for word in tokens]        #stemming
    tokens = [str(hash(word)) for word in tokens]                   #consider the hash code of the words will be useful for the next queries
    return tokens

In [59]:
def vocab(text):
    for word in set(text):                                          #create the vocabuary using the synopsis 
        vocabulary.append(word)

In [95]:
description = {}
vocabulary = []
num_pag = 20
for j in range(0,num_pag):
    for i in range((50*j)+1, (50*(j+1))+1):
        file = open( "pages_tsv/pages_tsv/page_" + str(j+1) + "/anime_" + str(i)+ ".tsv", "r", encoding = "utf-8")       
        anime = file.read()                                             
        file.close()
        anime = anime.split("\t")
        synopsis = anime[24]
        synopsis = text_mining(synopsis)
        vocab(synopsis)
        doc = "".join("documento "+str(i))
        description[doc] = synopsis
        title = anime[14]
        print(title)

animeStaff

Fullmetal Alchemist: Brotherhood
animeStaff

Gintama°
animeStaff

Shingeki no Kyojin Season 3 Part 2
animeStaff

Steins;Gate
animeStaff

Fruits Basket: The Final
animeStaff

Gintama'
animeStaff

Hunter x Hunter (2011)
animeStaff

Ginga Eiyuu Densetsu
animeStaff

Gintama': Enchousen
animeStaff

Gintama: The Final
animeStaff

Gintama.
animeStaff

3-gatsu no Lion 2nd Season
animeStaff

Koe no Katachi
animeStaff

Clannad: After Story
animeStaff

Gintama
animeStaff

Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare
animeStaff

Violet Evergarden Movie
animeStaff

Owarimonogatari 2nd Season
animeStaff

Shingeki no Kyojin: The Final Season
animeStaff

Code Geass: Hangyaku no Lelouch R2
animeStaff

Kimi no Na wa.
animeStaff

Gintama.: Shirogane no Tamashii-hen - Kouhan-sen
animeStaff

Shiguang Dailiren
animeStaff

Mob Psycho 100 II
animeStaff

Gintama.: Shirogane no Tamashii-hen
animeStaff

Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou
animeStaff

Kizumonogatari III

animeStaff

Hajime no Ippo: Champion Road
animeStaff

Hunter x Hunter: Greed Island Final
animeStaff

Re:Zero kara Hajimeru Isekai Seikatsu
animeStaff

Sennen Joyuu
animeStaff

Space☆Dandy 2nd Season
animeStaff

No Game No Life: Zero
animeStaff

Nodame Cantabile: Finale
animeStaff

Non Non Biyori Movie: Vacation
animeStaff

Ookami to Koushinryou
animeStaff

Stand By Me Doraemon 2
animeStaff

Youjo Senki Movie
animeStaff

Danshi Koukousei no Nichijou
animeStaff

Douluo Dalu: Haishen Zhi Guang
animeStaff

Douluo Dalu: Hanhai Qian Kun
animeStaff

Gyakkyou Burai Kaiji: Hakairoku-hen
animeStaff

IDOLiSH7: Third Beat!
animeStaff

Initial D Final Stage
animeStaff

Kuroshitsuji Movie: Book of the Atlantic
animeStaff

Major S3
animeStaff

Detective Conan: Episode One - The Great Detective Turned Small
animeStaff

Diamond no Ace: Act II
animeStaff

Hachimitsu to Clover II
animeStaff

Kuroko no Basket 2nd Season
animeStaff

Magi: The Kingdom of Magic
animeStaff

Major S1
animeStaff

Steins;Gate: 

animeStaff

Glass no Kamen (2005)
animeStaff

Koukyoushihen Eureka Seven
animeStaff

Kyou kara Ore wa!!
animeStaff

Love Live! Superstar!!
animeStaff

Black Clover
animeStaff

City Hunter 2
animeStaff

Girls & Panzer: Saishuushou Part 2
animeStaff

Kamisama Hajimemashita
animeStaff

Kill la Kill
animeStaff

Serial Experiments Lain
animeStaff

Shokugeki no Souma: San no Sara
animeStaff

Summer Wars
animeStaff

xxxHOLiC Shunmuki
animeStaff

Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku OVA
animeStaff

Paprika
animeStaff

Saint Seiya: Meiou Hades Juuni Kyuu-hen
animeStaff

Saiunkoku Monogatari 2nd Season
animeStaff

Tsumiki no Ie
animeStaff

Zombieland Saga: Revenge
animeStaff

BanG Dream! 3rd Season
animeStaff

Boku no Hero Academia the Movie 2: Heroes:Rising
animeStaff

Detective Conan Movie 10: Requiem of the Detectives
animeStaff

Eve no Jikan
animeStaff

Full Metal Panic? Fumoffu
animeStaff

Kara no Kyoukai 3: Tsuukaku Zanryuu
animeStaff

Kodomo no Omocha (TV)
animeStaff



animeStaff

Sekaiichi Hatsukoi 2
animeStaff

Sekaiichi Hatsukoi Movie: Yokozawa Takafumi no Baai
animeStaff

Tiger & Bunny
animeStaff

Wotaku ni Koi wa Muzukashii OVA
animeStaff

One Piece 3D2Y: Ace no shi wo Koete! Luffy Nakama Tono Chikai
animeStaff

One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken
animeStaff

Ookami to Koushinryou II: Ookami to Kohakuiro no Yuuutsu
animeStaff

There She Is!!
animeStaff

Vampire Hunter D (2000)
animeStaff

Zan Sayonara Zetsubou Sensei
animeStaff

Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. Movie
animeStaff

Berserk: Ougon Jidai-hen II - Doldrey Kouryaku
animeStaff

Bungou Stray Dogs Wan!
animeStaff

Bungou Stray Dogs: Dead Apple
animeStaff

Gake no Ue no Ponyo
animeStaff

Gakuen Babysitters
animeStaff

Gekkan Shoujo Nozaki-kun
animeStaff

Inochi no Tabekata
animeStaff

Kokoro ga Sakebitagatterunda.
animeStaff

Kono Oto Tomare!
animeStaff

Lupin III vs. Detective Conan: The Movie
animeStaff

Mahou Shoujo Lyrical Nan

animeStaff

New Initial D Movie: Legend 1 - Kakusei
animeStaff

Seitokai Yakuindomo Movie 2
animeStaff

Shadows House
animeStaff

Shaman King
animeStaff

Texhnolyze
animeStaff

The Disappearance of Conan Edogawa: The Worst Two Days in History
animeStaff

The Last: Naruto the Movie
animeStaff

Uramichi Oniisan
animeStaff

Akagami no Shirayuki-hime
animeStaff

Detective Conan OVA 10: Kid in Trap Island
animeStaff

Digimon Adventure
animeStaff

Digimon Adventure: Bokura no War Game!
animeStaff

"Haikyuu!!: vs. ""Akaten"""
animeStaff

Hibike! Euphonium Movie 2: Todoketai Melody
animeStaff

Lupin the Third: Mine Fujiko to Iu Onna
animeStaff

Aggressive Retsuko (ONA) 2nd Season
animeStaff

Blue Period
animeStaff

City Hunter '91
animeStaff

Claymore
animeStaff

Enen no Shouboutai: Ni no Shou
animeStaff

Free!: Take Your Marks
animeStaff

Genshiken 2
animeStaff

Golden Time
animeStaff

Hibike! Euphonium Movie 3: Chikai no Finale
animeStaff

Jinrui wa Suitai Shimashita
animeStaff

Kino no Tabi

### 2.1. Conjunctive query

In [61]:
vocabulary = list(set(vocabulary))                    #create the vocabulatory with non repetitions of words
file = open("vocabulary.txt", "w", encoding = "utf-8")
for word in vocabulary:
    file.write(str(hash(word)) +"\n")                 #create the file .txt that maps each word to a term_id using tha hash
file.close()                                          #function of python

### 2.1.1) Create your index!

In [62]:
inverted_index = {}
for term_id in vocabulary:
    for i in description:
        if term_id in description[i]:
            if term_id not in inverted_index:
                inverted_index[term_id] =  [i]
            else:
                inverted_index[term_id] = inverted_index[term_id] + [i]
print(inverted_index)

{'8060201492860552313': ['documento 171', 'documento 178', 'documento 217', 'documento 231', 'documento 379', 'documento 417', 'documento 426', 'documento 543', 'documento 544', 'documento 576', 'documento 681', 'documento 714', 'documento 745', 'documento 767', 'documento 844', 'documento 845', 'documento 871', 'documento 924', 'documento 946', 'documento 962'], '-3918677078797288957': ['documento 865'], '5935071079679112403': ['documento 270', 'documento 477', 'documento 941'], '-1906679295928973276': ['documento 432'], '-7403403440605085041': ['documento 394'], '3030073481205858125': ['documento 953'], '-2456254020771204256': ['documento 1'], '-1479933760261539297': ['documento 12', 'documento 54', 'documento 81', 'documento 93', 'documento 111', 'documento 157', 'documento 206', 'documento 223', 'documento 271', 'documento 275', 'documento 302', 'documento 323', 'documento 378', 'documento 384', 'documento 386', 'documento 448', 'documento 457', 'documento 476', 'documento 486', 'd

### 2.1.2) Execute the query

In [90]:
def search_word(query):
    d = []
    query = set(text_mining(query))
    for word in query:
        d.append(inverted_index[word])
    print(d)
    for i in range(len(d)-1):
        s = set(d[i]).intersection(set(d[i+1]))
    return  s
        

In [91]:
query = "saiyan race"
search_word(query)

[['documento 91', 'documento 100', 'documento 119', 'documento 141', 'documento 143', 'documento 147', 'documento 173', 'documento 184', 'documento 235', 'documento 236', 'documento 240', 'documento 260', 'documento 271', 'documento 297', 'documento 314', 'documento 333', 'documento 365', 'documento 371', 'documento 378', 'documento 391', 'documento 397', 'documento 401', 'documento 452', 'documento 457', 'documento 494', 'documento 501', 'documento 504', 'documento 515', 'documento 518', 'documento 529', 'documento 560', 'documento 603', 'documento 608', 'documento 613', 'documento 661', 'documento 680', 'documento 694', 'documento 726', 'documento 732', 'documento 749', 'documento 750', 'documento 774', 'documento 798', 'documento 802', 'documento 811', 'documento 831', 'documento 898', 'documento 910', 'documento 928'], ['documento 365', 'documento 401']]
{'documento 401', 'documento 365'}
